In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# setting plot size
%pylab inline
pylab.rcParams['figure.figsize'] = (16, 9)

In [2]:
%run ./feature_engineering.py
%run ./utils.py

# 1. ALL DATA

## 1.0. IMPORTING/CLEANING DATA

In [3]:
import time
start = time.time()
# i want the list of the files from there 
import os
tickers = os.listdir("/Users/edouardcuny/Desktop/quant/Carmela/data2/")
l = []
for ticker in tickers:
    if 'csv' in ticker:
        df = build_df(ticker)
        df.drop(df.columns[0], axis=1,inplace=True)
        l.append(df)

df = pd.concat(l, axis=0)
# df.to_csv("/Users/edouardcuny/Desktop/stocks.csv")

# removing rows w/ infinite
df.replace([np.inf, -np.inf], np.nan,inplace=True)

# removing rows w/ NaN i.e. first rows and last rows based on window_size
df.dropna(inplace=True)

stop = time.time()
print 'temps de création du DataFrame : ' + str(round(stop - start))

dropped NA : 4653 >> 4613
dropped NA : 4144 >> 4120
dropped NA : 4653 >> 4616
dropped NA : 4218 >> 4187
dropped NA : 4653 >> 4614
dropped NA : 7513 >> 6585
dropped NA : 6273 >> 5773
dropped NA : 4653 >> 4616


/Users/edouardcuny/Desktop/quant/Carmela/feature_engineering.py:253: RuntimeWarning: Mean of empty slice.
  fup = lambda x : x[x>0].mean()


dropped NA : 4653 >> 4608
dropped NA : 4653 >> 4613
dropped NA : 4653 >> 4600


/Users/edouardcuny/Desktop/quant/Carmela/feature_engineering.py:254: RuntimeWarning: Mean of empty slice.
  fdown = lambda x : np.abs(x[x<0].mean())


dropped NA : 6998 >> 6625
dropped NA : 4653 >> 4447
dropped NA : 4653 >> 4613
dropped NA : 4652 >> 2231
dropped NA : 4653 >> 4615
dropped NA : 4653 >> 4483
dropped NA : 656 >> 656
dropped NA : 3019 >> 3016
dropped NA : 4653 >> 4609
dropped NA : 4653 >> 4615
dropped NA : 4653 >> 4610
dropped NA : 4653 >> 4615
dropped NA : 4653 >> 4607
dropped NA : 4642 >> 3805
dropped NA : 4653 >> 4614
dropped NA : 22 >> 22
dropped NA : 4653 >> 4610
dropped NA : 4510 >> 4471
dropped NA : 4653 >> 4614
72.4118528366


## 1.1. ML

In [4]:
X_train,X_test,Y_train,Y_test = split_train_test(df,date="2012-12-31")

dropped extreme values : 126954 >> 107402
dropped NA : 107402 >> 107402


/Users/edouardcuny/.pyenv/versions/2.7.10/envs/ml/lib/python2.7/site-packages/pandas/core/indexing.py:601: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [5]:
print X_train.shape
print X_test.shape
print Y_train.shape
print Y_test.shape

(78369, 43)
(29033, 43)
(78369,)
(29033,)


In [15]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train,Y_train)
Y_pred = pd.Series(clf.predict(X_test))
Y_pred.index = Y_test.index

In [16]:
print len(Y_test[Y_test!=0])
print len(Y_test)
print np.float64(len(Y_test[Y_test!=0]))/len(Y_test)
print
print len(Y_pred[Y_pred!=0])
print len(Y_pred)
print np.float64(len(Y_pred[Y_pred!=0]))/len(Y_pred)


14395
29033
0.495815106947

11329
29033
0.390211139049
